# Plotting example images from Justdiggit drone data published on Google Cloud along with their annotations

credits: Tim Broadhurst

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
29 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 27.5 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.5_amd64.deb ...
Unpacking gcsfuse (0.41.5) ...
Setting up gcsfuse (0.41.5) ...


In [3]:
!mkdir -p justdiggit-drone-data
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 justdiggit-drone justdiggit-drone-data

2022/08/11 02:06:42.546456 Start gcsfuse/0.41.5 (Go version go1.18.4) for app "" using mount point: /content/justdiggit-drone-data
2022/08/11 02:06:42.562292 Opening GCS connection...
2022/08/11 02:06:44.938503 Mounting file system "justdiggit-drone"...
2022/08/11 02:06:44.970034 File system has been successfully mounted.


In [7]:
DATA_path = "/content/justdiggit-drone-data/label_sample"

In [8]:
import json

In [9]:
f = open(DATA_path +'/Annotations_trees_only.json')

data = json.load(f)

In [10]:
print(data.keys())

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])


In [11]:
print(data['annotations'][1])

{'id': 2, 'image_id': 1, 'segmentation': [[275, 323, 274, 324, 273, 324, 272, 324, 271, 324, 271, 325, 272, 326, 272, 327, 271, 328, 271, 329, 271, 330, 272, 331, 271, 332, 270, 332, 269, 332, 268, 332, 267, 332, 266, 333, 266, 334, 265, 335, 264, 336, 263, 336, 262, 335, 261, 335, 260, 336, 259, 337, 258, 338, 258, 339, 257, 340, 256, 340, 255, 340, 254, 340, 253, 340, 252, 340, 251, 341, 250, 341, 249, 342, 248, 343, 247, 344, 246, 345, 245, 346, 244, 347, 243, 348, 242, 349, 241, 350, 240, 350, 239, 350, 239, 351, 239, 352, 238, 353, 237, 353, 236, 353, 235, 354, 234, 355, 233, 356, 232, 356, 231, 357, 230, 358, 229, 358, 228, 358, 227, 358, 226, 358, 225, 358, 224, 358, 223, 359, 222, 359, 221, 360, 220, 360, 219, 360, 218, 361, 217, 361, 216, 361, 215, 362, 214, 362, 213, 363, 213, 364, 213, 365, 213, 366, 213, 367, 213, 368, 213, 369, 213, 370, 213, 371, 213, 372, 213, 373, 213, 374, 213, 375, 212, 376, 211, 377, 210, 378, 209, 378, 208, 379, 207, 380, 206, 380, 205, 380, 204, 38

In [12]:
print(data['images'][0])

{'id': 1, 'file_name': 'bumila_year_2018_tile_6000_12000.png', 'height': 1000, 'width': 1000, 'license': 1}


In [13]:
images = list((x['file_name'],x['id']) for x in data['images'])
print(images[:3])

[('bumila_year_2018_tile_6000_12000.png', 1), ('bumila_year_2018_tile_6000_6000.png', 2), ('bumila_year_2018_tile_5000_7000.png', 3)]


In [14]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
import os.path
from os import path

DRIVE_path = '/content/drive/MyDrive/AI4Trees/'
if path.exists(DRIVE_path + 'example_images_with_annotations') == False:
  os.mkdir(DRIVE_path + 'example_images_with_annotations')

In [23]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import numpy as np

num_images_to_plot = 5 # it takes 30 mins to plot all ~1100 images

for idx, file_id_tuple in enumerate(images):
  if idx == num_images_to_plot:
    break
  print(file_id_tuple)
  img = file_id_tuple[0]
  id = file_id_tuple[1]
  img_path = "/content/justdiggit-drone-data/label_sample/" + img
  print(img_path)
  img_array = cv2.imread(img_path)
  fig = plt.figure(figsize=(12,16))
  ax = fig.add_subplot(1,1,1)
  ax.imshow(img_array)

  for i,annot in enumerate(data['annotations']):
    if annot['image_id'] == id:
      # put mask on the image
      segmentation = data['annotations'][i]['segmentation']
      for seg in segmentation:
        x_list = np.array(seg[0::2])
        y_list = np.array(seg[1::2])
        ax.fill(x_list, y_list, alpha=0.5, fc='r', ec='none')
      #create the rectangle object and add it to the image
      bbox = annot['bbox']
      x1, y1, width, height = bbox
      rect = Rectangle((x1, y1), width, height, fill=False, color='red')
      ax.add_patch(rect)
  save_path = DRIVE_path + 'example_images_with_annotations/'
  figname = 'fig_{}.png'.format(img.replace('.png', ''))
  dest = os.path.join(save_path, figname)
  plt.savefig(dest)  # write image to file
  plt.cla()
  plt.close(fig)

('bumila_year_2018_tile_6000_12000.png', 1)
/content/justdiggit-drone-data/label_sample/bumila_year_2018_tile_6000_12000.png
('bumila_year_2018_tile_6000_6000.png', 2)
/content/justdiggit-drone-data/label_sample/bumila_year_2018_tile_6000_6000.png
('bumila_year_2018_tile_5000_7000.png', 3)
/content/justdiggit-drone-data/label_sample/bumila_year_2018_tile_5000_7000.png
('bumila_year_2018_tile_5000_6000.png', 4)
/content/justdiggit-drone-data/label_sample/bumila_year_2018_tile_5000_6000.png
('bumila_year_2018_tile_2000_10000.png', 5)
/content/justdiggit-drone-data/label_sample/bumila_year_2018_tile_2000_10000.png
